# Limpieza de un CSV con data-cleaner

Se utilizan **data-cleaner** y **pandas** para codificar la limpieza de los datos de un archivo CSV. Primero se realiza una exploración de la tabla aplicando algunas reglas de limpieza y comprobando el resultado generado. Cuando este resultado es satisfactorio, se agrega la regla de limpieza a la lista codificada que luego se utilizará para generar la versión limpia del archivo.

## Inicio

In [6]:
from data_cleaner import DataCleaner

In [7]:
input_path = "audiencias-raw.csv"
output_path = "audiencias-clean.csv"

In [8]:
dc = DataCleaner(input_path)

## Exploración y descubrimiento

In [4]:
dc.df

,lugar,sintesis,hora_audiencia,objeto,participantes,fecha_solicitud,id_dependencia,dependencia,fecha_audiencia,interes_invocado,datos_solicitante,sujeto_obligado,datos_representado,estado,id
0,Casa de Gobierno,Repasaron los principales temas de la agenda p...,20:00,Temas de gestión provincial,"Dr. Abal Medina, Juan Manuel, Oscar Mario Jorge.",12-11-2013,43,Jefatura de Gabinete de Ministros,12-11-2013,Colectivo Organismo Estatal,"Jorge, Oscar Mario (Gobernador de La Pampa) DN...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",GOBERNACION DE LA PAMPA (MINISTRO DE GOBIERNO),Realizada,61419
1,Casa de Gobierno,Antonio Vulcano y Martín Reibel expusieron los...,12:00,Analizaron políticas del organismo,"Dr. Abal Medina, Juan Manuel, Antonio Vulcano ...",12-11-2013,43,Jefatura de Gabinete de Ministros,12-11-2013,Colectivo Organismo Estatal,"VULCANO, Antonio (Presidente de la Agencia de ...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",Agencia de Administración de Bienes del Estado,Realizada,61420
2,Casa de Gobierno,Analizaron los principales temas de agenda y p...,19:00,Temas de agenda en la ciudad,"Dr. Abal Medina, Juan Manuel y Dip. Juan Cabandié",11-11-2013,43,Jefatura de Gabinete de Ministros,11-11-2013,Colectivo Particular Interesado,"Cabandie, Juan (Diputado Nacional) DNI 18842578","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",NaN,Realizada,61418
3,Casa de Gobierno,Repasaron los principales temas de gestión de ...,18:00,Temas de gestión provincial,"Dr. Abal Medina, Juan Manuel y Gob. Lucía Corp...",11-11-2013,43,Jefatura de Gabinete de Ministros,11-11-2013,Colectivo Particular Interesado,"CORPACCI, LUCIA (Gobernadora de la Provincia ...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",NaN,Realizada,61417
4,Casa de Gobierno,Repasaron la agenda legislativa de los próximo...,12:00,Temas de agenda legislativa,"Dr. Abal Medina, Juan Manuel, Dip. Julián Domi...",11-11-2013,43,Jefatura de Gabinete de Ministros,11-11-2013,Colectivo Particular Interesado,"Dominguez, Julián (Presidente de la Camara de ...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",NaN,Realizada,61416
5,Casa de Gobierno,Temas de gestión,17:30,Temas de gestión,"Dr. Abal Medina, Juan Manuel y Dr. Marcelo A. ...",08-11-2013,43,Jefatura de Gabinete de Ministros,08-11-2013,Colectivo Organismo Estatal,"BUFACCHI, Marcelo (Coordinador de Gabinete de ...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",Jefatura de Gabinete de Ministros,Realizada,61415
6,Casa de Gobierno,Firmaron un acuerdo para que profesionales arg...,10:30,Firma de Convenios,"Dr. Abal Medina, Juan Manuel y Sr. Bishwapriya...",08-11-2013,43,Jefatura de Gabinete de Ministros,08-11-2013,Colectivo Particular Interesado,"Sanyal, Bishwapriya (Director del Programa Esp...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",NaN,Realizada,61414
7,Casa de Gobierno,Repasaron hoy la agenda de obras que se llevan...,17:00,Temas de gestión,"Dr. Abal Medina, Juan Manuel y Sr. Eduardo Acc...",07-11-2013,43,Jefatura de Gabinete de Ministros,07-11-2013,Colectivo Particular Interesado,"ACCASTELLO, EDUARDO (Intendente de Villa Maria...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",NaN,Realizada,61413
8,Casa de Gobierno,Analizaron con el Senador la agenda legislativ...,11:00,Temas de Gestión,"Dr. Abal Medina, Juan Manuel y Miguel Pichetto",06-11-2013,43,Jefatura de Gabinete de Ministros,06-11-2013,Colectivo Particular Interesado,"Pichetto, Miguel (Senador Nacional) LE 12555987","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",NaN,Realizada,61412
9,Casa de Gobierno,En en encuentro se hizo un repaso de la situac...,20:00,Temas de gestión,"Dr. Abal Medina, Juan Manuel, Sr. Martin Buzzi...",05-11-2013,43,Jefatura de Gabinete de Ministros,05-11-2013,Colectivo Organismo Estatal,"BUZZI, Martín (Gobernador de la Provincia del ...","ABAL MEDINA, Juan Manuel Cargo: Jefe de Gabine...",Gobernacion de Chubut (Gobernador),Realizada,61411


In [5]:
dc.nombre_propio("dependencia").tail()

/Users/abenassi/anaconda/envs/data_cleaner/lib/python2.7/site-packages/unidecode/__init__.py:46: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  _warn_if_not_unicode(string)


60595    Ministerio de Justicia y Derechos Humanos Proc...
60596    Ministerio de Justicia y Derechos Humanos Proc...
60597    Ministerio de Justicia y Derechos Humanos Proc...
60598    Ministerio de Justicia y Derechos Humanos Proc...
60599    Ministerio de Ciencia, Tecnología E Innovación...
Name: dependencia, dtype: object

## Reglas de limpieza codificadas

In [9]:
rules = [
    {
        "nombre_propio": [
            {"field": "dependencia", "keep_original": False}
        ]
    },
    {
        "string": [
            {"field": "lugar", "keep_original": False},
            {"field": "sintesis", "keep_original": False},
            {"field": "objeto", "keep_original": False},
            {"field": "dependencia", "keep_original": False},
            {"field": "interes_invocado", "keep_original": False},
            {"field": "datos_representado", "keep_original": False}
        ]
    },
    {
        "fecha_separada": [
            {"fields": [["fecha_audiencia", "DD-MM-YYYY"],
                        ["hora_audiencia", "HH:mm"]],
             "new_field_name": "audiencia",
             "keep_original": True}
        ]
    },
    {
        "fecha_simple": [
            {"field": "fecha_solicitud",
             "time_format": "DD-MM-YYYY",
             "keep_original": False}
        ]
    },
    {
        "string_simple_split": [
            {"field": "sujeto_obligado",
             "separators": [", Cargo:", "Cargo:"],
             "new_field_names": ["nombre", "cargo"],
             "keep_original": True}
        ]
    },
    {
        "nombre_propio": [
            {"field": "sujeto_obligado_nombre", "keep_original": False},
            {"field": "sujeto_obligado_cargo", "keep_original": False}
        ]
    },
    {
        "string": [
            {"field": "sujeto_obligado_nombre", "keep_original": False},
            {"field": "sujeto_obligado_cargo", "keep_original": False}
        ]
    },
    {
        "string_peg_split": [
            {"field": "datos_solicitante",
             "grammar": """
            allowed_char = anything:x ?(x not in '1234567890() ')
            nombre = ~('DNI') <allowed_char+>:n ws -> n.strip()
            number = <digit+>:num -> int(num)

            nom_comp = <nombre+>:nc -> nc.strip()
            cargo = '(' <nombre+>:c ')' -> c.strip()
            dni = ','? ws 'DNI' ws number:num -> num

            values = nom_comp:n ws cargo?:c ws dni?:d ws anything* -> [n, c, d]
             """,
             "new_field_names": ["nombre", "cargo", "dni"],
             "keep_original": True}
        ]
    },
    {
        "nombre_propio": [
            {"field": "datos_solicitante_nombre", "keep_original": False}
        ]
    },
    {
        "string": [
            {"field": "datos_solicitante_nombre", "keep_original": False}
        ]
    },
]

In [10]:
dc.clean_file(rules, output_path)

/Users/abenassi/anaconda/envs/data_cleaner/lib/python2.7/site-packages/unidecode/__init__.py:46: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  _warn_if_not_unicode(string)
